# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#!jupyter nbextension enable --py gmaps
#%matplotlib inline
#%matplotlib notebook
#%matplotlib ipympl

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json
import ipywidgets as widgets

from pygments import highlight, lexers, formatters
def ppj (json_obj, dbug=False):
    colormej = json.dumps(json_obj, sort_keys=True, indent=4)
    colorful_json = highlight(colormej, lexers.JsonLexer(), formatters.TerminalFormatter())
    if dbug: print(colorful_json)

# Import API key
orig_wd = os.getcwd()
os.chdir(os.path.join('..','..','..','..','..'))
from config import gkey
os.chdir(orig_wd)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv("cities.csv")
print(len(cities_df))
cities_df.head(2)

860


,Unnamed: 0,City,Date,Country,Humidity,Max Temperature,Wind Speed,Conditions,Cond_description,Latitude,Longitude,Cloudiness
0,0,São Félix do Xingu,1591115011,BR,60,86.22,3.67,Rain,light rain,-6.64,-51.99,100
1,1,Touros,1591114792,BR,66,86.00,9.17,Clouds,scattered clouds,-5.20,-35.46,40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=gkey)
#gkey

In [5]:
#humidity
heatmap = gmaps.figure(map_type='SATELLITE')
heatmap_layer = gmaps.heatmap_layer(cities_df[["Latitude", "Longitude"]], weights=cities_df["Humidity"],
                                    dissipating=False, max_intensity=200,
                                    point_radius=2)

heatmap.add_layer(heatmap_layer)
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [6]:

ideal_cities_df = cities_df.loc[(cities_df["Max Temperature"] <= 89) & (cities_df["Max Temperature"] > 70) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]
print(f"There are {len(ideal_cities_df)} cities with optimal conditions, from the inital list of {len(cities_df)} cities ")

There are 19 cities with optimal conditions, from the inital list of 860 cities 


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#ideal_cities_df['Hotel'] = [ '' for row in ideal_cities_df['City']]
ideal_cities_df.reset_index(inplace=True)


In [65]:

def crash_pad (loca_df, qterm, dist=5000, locatype='hotel'):
    #for i in range(len(loca_df.head(1))):
    hotel_name = []
    hotel_lat = []
    hotel_lon = []
    hotel_rating = []
    for i, row in loca_df.iterrows():
        hotel = {}
        #print(loca_df.loc[i, 'City']) 
        query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        query_url += f"?location={loca_df.loc[i, 'Latitude']},{loca_df.loc[i, 'Longitude']}&radius={dist}&type={locatype}&keyword={qterm}&key={gkey}"
        googlereturn = requests.get(query_url)
        gjson = googlereturn.json()['results']
        if len(gjson) > 0:
            #print(f"{i} {gjson[0]['name']}")
            hotel_name.append(gjson[0]['name'])
            hotel_lat.append(f"{gjson[0]['geometry']['location']['lat']}")
            hotel_lon.append(f"{gjson[0]['geometry']['location']['lng']}")
            hotel_rating.append(gjson[0]['rating'])
        else:
            #print(f"{i} No hotel was found near [{loca_df.loc[i, 'City']}] [{len(gjson)}]")
            hotel_name.append('')
            hotel_lat.append('')
            hotel_lon.append('')
            hotel_rating.append('')
    loca_df['Hotel_Name'] = hotel_name
    loca_df['Hotel_Lat'] = hotel_lat
    loca_df['Hotel_lon'] = hotel_lon
    loca_df['Hotel_Rating'] = hotel_rating
    return loca_df

In [66]:
ideal_cities_df.head(2)

,index,Unnamed: 0,City,Date,Country,Humidity,Max Temperature,Wind Speed,Conditions,Cond_description,Latitude,Longitude,Cloudiness,Hotel Name,Hotel Geo,Hotel Rating,Hotel_Name,Hotel_Geo,Hotel_Rating
0,9,9,Kumul,1591114769,CN,15,83.71,6.44,Clear,clear sky,42.80,93.45,0,Back Hotel,"(42.844404, 93.49309)",0,Haoli Business Hotel,"(42.830214, 93.503838)",0
1,60,60,Mulegé,1591115015,MX,37,88.70,4.72,Clear,clear sky,26.88,-111.98,0,Cuesta Real Hotel,"(26.8977357, -111.9652359)",3.7,Cuesta Real Hotel,"(26.8977357, -111.9652359)",3.7


In [67]:

returned = crash_pad(ideal_cities_df, 'hotel')
returned

0 Haoli Business Hotel
1 Cuesta Real Hotel
2 Hotel Litu
3 Haji Abdul Samad Hotel
4 Hotel Mathilda
5 Hotel BasTau
6 No hotel was found near [Sangīn] [0]
7 ilman Yousafzai
8 Hotel National
9 Rajdhani Hotel
10 Shada Collection Mina Hotel
11 Coral Coast Tourist Park
12 Ormara Beach Resorts/ Bolan Safari
13 Parque do Avestruz Hotel Fazenda
14 No hotel was found near [Tiznit Province] [0]
15 JinJiang International Hotel Urumqi
16 Kempinski Hotel Yinchuan
17 Hakusembe River Lodge
18 Hotel Las Águilas


C:\Users\SeaLord\anaconda37\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\SeaLord\anaconda37\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\SeaLord\anaconda37\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

,index,Unnamed: 0,City,Date,Country,Humidity,Max Temperature,Wind Speed,Conditions,Cond_description,...,Longitude,Cloudiness,Hotel Name,Hotel Geo,Hotel Rating,Hotel_Name,Hotel_Geo,Hotel_Rating,Hotel_Lat,Hotel_lon
0,9,9,Kumul,1591114769,CN,15,83.71,6.44,Clear,clear sky,...,93.45,0,Back Hotel,"(42.844404, 93.49309)",0,Haoli Business Hotel,"(42.830214, 93.503838)",0,42.830214,93.503838
1,60,60,Mulegé,1591115015,MX,37,88.70,4.72,Clear,clear sky,...,-111.98,0,Cuesta Real Hotel,"(26.8977357, -111.9652359)",3.7,Cuesta Real Hotel,"(26.8977357, -111.9652359)",3.7,26.8977357,-111.9652359
2,113,113,Eenhana,1591115018,NaN,18,82.40,4.70,Clear,clear sky,...,16.33,0,Hotel Litu,"(-17.4879171, 16.3343763)",3.8,Hotel Litu,"(-17.4879171, 16.3343763)",3.8,-17.4879171,16.3343763
3,154,154,Pishin,1591114996,PK,13,80.65,4.09,Clear,clear sky,...,67.00,0,Haji Abdul Samad Hotel,"(30.587339, 67.00463719999999)",2,Haji Abdul Samad Hotel,"(30.587339, 67.00463719999999)",2,30.587339,67.00463719999999
4,197,197,Maumere,1591115023,ID,72,78.82,5.23,Clear,clear sky,...,122.21,0,Hotel Mathilda,"(-8.6118572, 122.2083548)",4.4,Hotel Mathilda,"(-8.6118572, 122.2083548)",4.4,-8.6118572,122.2083548
5,205,205,Zhangaözen,1591115024,KZ,21,85.35,5.39,Clear,clear sky,...,52.80,0,Hotel BasTau,"(43.3374375, 52.8571812)",4.4,Hotel BasTau,"(43.3374375, 52.8571812)",4.4,43.3374375,52.8571812
6,211,211,Sangīn,1591115025,AF,10,88.03,4.70,Clear,clear sky,...,64.84,0,,,,,,,,
7,247,247,Nushki,1591115027,PK,10,85.80,3.60,Clear,clear sky,...,66.02,0,ilman Yousafzai,"(29.5563397, 66.0293834)",0,ilman Yousafzai,"(29.5563397, 66.0293834)",0,29.5563397,66.0293834
8,270,270,Ambilobe,1591114777,MG,82,73.17,0.94,Clear,clear sky,...,49.05,0,Hotel National,"(-13.1958827, 49.0492968)",3.5,Hotel National,"(-13.1958827, 49.0492968)",3.5,-13.1958827,49.0492968
9,347,347,Lakhipur,1591115035,IN,85,78.40,4.16,Clear,clear sky,...,90.30,0,Rajdhani Hotel,"(26.0314642, 90.30304840000001)",3.8,Rajdhani Hotel,"(26.0314642, 90.30304840000001)",3.8,26.0314642,90.30304840000001


In [68]:
hotel_df = returned.loc[returned["Hotel_Name"] != ""]
hotel_df.reset_index(inplace=True)
print(len(hotel_df))
hotel_df.head(3)

17


,level_0,index,Unnamed: 0,City,Date,Country,Humidity,Max Temperature,Wind Speed,Conditions,...,Longitude,Cloudiness,Hotel Name,Hotel Geo,Hotel Rating,Hotel_Name,Hotel_Geo,Hotel_Rating,Hotel_Lat,Hotel_lon
0,0,9,9,Kumul,1591114769,CN,15,83.71,6.44,Clear,...,93.45,0,Back Hotel,"(42.844404, 93.49309)",0,Haoli Business Hotel,"(42.830214, 93.503838)",0,42.830214,93.503838
1,1,60,60,Mulegé,1591115015,MX,37,88.70,4.72,Clear,...,-111.98,0,Cuesta Real Hotel,"(26.8977357, -111.9652359)",3.7,Cuesta Real Hotel,"(26.8977357, -111.9652359)",3.7,26.8977357,-111.9652359
2,2,113,113,Eenhana,1591115018,NaN,18,82.40,4.70,Clear,...,16.33,0,Hotel Litu,"(-17.4879171, 16.3343763)",3.8,Hotel Litu,"(-17.4879171, 16.3343763)",3.8,-17.4879171,16.3343763


In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = []
locations = []
for row in hotel_df.itertuples():
    #print(row)
    template = f"<dl><dt>Name</dt><dd>{row.Hotel_Name}</dd><dt>Rating</dt><dd>{row.Hotel_Rating}</dd><dt>City</dt><dd>{row.City}</dd><dt>Country</dt><dd>{row.Country}</dd></dl>"
    hotel_info.append(template)
    locations.append((float(row.Hotel_Lat),float(row.Hotel_lon)))
print(f"{hotel_info}\n\n {locations}")


['<dl><dt>Name</dt><dd>Haoli Business Hotel</dd><dt>Rating</dt><dd>0</dd><dt>City</dt><dd>Kumul</dd><dt>Country</dt><dd>CN</dd></dl>', '<dl><dt>Name</dt><dd>Cuesta Real Hotel</dd><dt>Rating</dt><dd>3.7</dd><dt>City</dt><dd>Mulegé</dd><dt>Country</dt><dd>MX</dd></dl>', '<dl><dt>Name</dt><dd>Hotel Litu</dd><dt>Rating</dt><dd>3.8</dd><dt>City</dt><dd>Eenhana</dd><dt>Country</dt><dd>nan</dd></dl>', '<dl><dt>Name</dt><dd>Haji Abdul Samad Hotel</dd><dt>Rating</dt><dd>2</dd><dt>City</dt><dd>Pishin</dd><dt>Country</dt><dd>PK</dd></dl>', '<dl><dt>Name</dt><dd>Hotel Mathilda</dd><dt>Rating</dt><dd>4.4</dd><dt>City</dt><dd>Maumere</dd><dt>Country</dt><dd>ID</dd></dl>', '<dl><dt>Name</dt><dd>Hotel BasTau</dd><dt>Rating</dt><dd>4.4</dd><dt>City</dt><dd>Zhangaözen</dd><dt>Country</dt><dd>KZ</dd></dl>', '<dl><dt>Name</dt><dd>ilman Yousafzai</dd><dt>Rating</dt><dd>0</dd><dt>City</dt><dd>Nushki</dd><dt>Country</dt><dd>PK</dd></dl>', '<dl><dt>Name</dt><dd>Hotel National</dd><dt>Rating</dt><dd>3.5</dd><d

In [100]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
        locations,
        info_box_content=hotel_info
    )

heatmap.add_layer(marker_layer)
# Display figure
heatmap

Figure(layout=FigureLayout(height='420px'))